## Add Table to a Hosted Feature Service from Template


In [1]:
# Import the ArcGIS API for Python
from arcgis.gis import GIS
# Import the ArcGIS API for Python FeatureLayerCollection manager
from arcgis.features import FeatureLayerCollection
# handy package for copying nested data
import copy

# Connect to the current AGO organization. In ArcGIS Online notebooks, "home" will use the current credentials.
gis = GIS("home")

You are logged on as Mukwonago_Consultant with an administrator role, proceed with caution.


### ItemID arguments

In [2]:
# Read in the itemID for the feature service we want to add the table to
target_itemid = "0b83556135ac421f8b89495208b6edee"

# Read in the itemID for the feature service we want to use as a template
source_itemid = "00f6ac17b10843fa9f82b50b622cc67c"

# Get the items
target_item = gis.content.get(target_itemid)
source_item = gis.content.get(source_itemid)

# Convert to Feature Layer Collections
target_flc = FeatureLayerCollection.fromitem(target_item)
source_flc = FeatureLayerCollection.fromitem(source_item)

### Get the definition from the template

In [3]:
source_table = source_flc.tables[0]

print(source_table)

<Table url:"https://services6.arcgis.com/3u5nlx5f2iT1o0oh/arcgis/rest/services/Tree_Inspection/FeatureServer/0">


### Convert the table properties to a dictionary

In [4]:
tbl_def = dict(source_table.properties)
print(tbl_def)

{'currentVersion': 11.5, 'id': 0, 'name': 'Tree Inspection', 'type': 'Table', 'serviceItemId': '00f6ac17b10843fa9f82b50b622cc67c', 'cacheMaxAge': 30, 'displayField': 'OBJECTID', 'description': '', 'copyrightText': '', 'defaultVisibility': True, 'editingInfo': {'lastEditDate': 1771995941662, 'schemaLastEditDate': 1771995941646, 'dataLastEditDate': 1771995941646}, 'relationships': [], 'isDataVersioned': False, 'hasContingentValuesDefinition': False, 'supportsAppend': True, 'supportsCalculate': True, 'supportsASyncCalculate': True, 'supportsTruncate': True, 'supportsAttachmentsByUploadId': True, 'supportsAttachmentsResizing': True, 'supportsRollbackOnFailureParameter': True, 'supportsStatistics': True, 'supportsExceedsLimitStatistics': True, 'supportsAdvancedQueries': True, 'supportsValidateSql': True, 'supportsCoordinatesQuantization': True, 'supportsLayerOverrides': True, 'supportsTilesAndBasicQueriesMode': True, 'supportsFieldDescriptionProperty': True, 'supportsQuantizationEditMode': 

### Step 1: Add the table definition to the feature service

In [5]:
target_flc.manager.add_to_definition({"tables": [tbl_def]})

{'success': True, 'layers': [{'name': 'Tree Inspection', 'id': 1}]}

### Step 2: Update the data JSON of the feature service so that the table actually appears in the AGOL UI

In [6]:
# Get data JSONs for both source and target
source_data = source_item.get_data()
target_data = target_item.get_data() or {}

# Ensure tables array exists in target data
if "tables" not in target_data:
    target_data["tables"] = []

# Copy the full table entry from the source data
source_table_entry = copy.deepcopy(source_data["tables"][0])

### Set the index for the new table to be added

In [10]:
source_table_entry["id"] = 1

### Perform the update in memory

In [11]:
# Append
target_data["tables"].append(source_table_entry)
print(target_data)

{'layers': [{'id': 0, 'popupInfo': {'popupElements': [{'type': 'fields'}, {'type': 'attachments', 'displayType': 'auto'}], 'showAttachments': True, 'fieldInfos': [{'fieldName': 'OBJECTID', 'isEditable': False, 'label': 'OBJECTID', 'visible': False}, {'fieldName': 'CommonName', 'isEditable': True, 'label': 'Common Name', 'tooltip': '', 'visible': True}, {'fieldName': 'Diameter', 'format': {'digitSeparator': True, 'places': 0}, 'isEditable': True, 'label': 'Diameter', 'tooltip': '', 'visible': True}, {'fieldName': 'Valuation', 'format': {'digitSeparator': True, 'places': 2}, 'isEditable': True, 'label': 'Valuation', 'tooltip': '', 'visible': True}], 'title': 'Trees'}, 'layerDefinition': {'drawingInfo': {'renderer': {'type': 'simple', 'symbol': {'type': 'CIMSymbolReference', 'symbol': {'type': 'CIMPointSymbol', 'symbolLayers': [{'type': 'CIMVectorMarker', 'enable': True, 'anchorPoint': {'x': 4.689134360376102e-06, 'y': 4.706369600645151e-06, 'z': 0}, 'anchorPointUnits': 'Relative', 'domin

### Push the update to the HFL

In [12]:
# Push update to the item
target_item.update(data=target_data)

True